In [1]:
# create animation on real-time updating data

import pandas as pd
import numpy as np
import traceback as tb
import time

import plotly.graph_objects as go


# Linear regression (on 2-variable function)

In [5]:
# XOR function -- along with NAND (Not-and), OR, AND
# truth tables


# input variable values on training data

train_data = np.array(
    [
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]])

# output variable values on training data -- pick one to train a specific function

# for this example, there is NO separate testing data (input only has finite (4) possibilities)

# it is a huge blow to perceptron based method as it cannot even behave on training data (untrainable)!

# note this is an inheritent issue of the model, not an issue of a particular training method.

target_xor = np.array(
    [
        [0],
        [1],
        [1],
        [0]])

target_nand = np.array(
    [
        [1],
        [1],
        [1],
        [0]])

target_or = np.array(
    [
        [0],
        [1],
        [1],
        [1]])

target_and = np.array(
    [
        [0],
        [0],
        [0],
        [1]])

target_first = np.array(
    [
        [1],
        [1],
        [0],
        [0]
        
    ])

## Perceptron class

In [3]:
from itertools import cycle

# original code is See https://towardsdatascience.com/how-neural-networks-solve-the-xor-problem-59763136bdd7
# we added visualization
color_scale = [[0., 'gold'], [0.5, 'mediumturquoise'], [1., 'lightsalmon']]
class Perceptron:
    """
    Create a perceptron.
    train_data: A 4x2 matrix with the input data.
    target: A 4x1 matrix with the perceptron's expected outputs
    lr: the learning rate. Defaults to 0.01
    input_nodes: the number of nodes in the input layer of the perceptron.
        Should be equal to the second dimension of train_data.
    """

    def __init__(self, train_data, target, lr=0.01, input_nodes=2, h=0.01, fig=None):
        self.train_data = train_data
        self.target = target
        self.lr = lr
        self.input_nodes = input_nodes

        # randomly initialize the weights and set the bias to -1.
        self.w = np.random.uniform(size=self.input_nodes)
        self.b = -1

        # node_val hold the values of each node at a given point of time.
        self.node_val = np.zeros(self.input_nodes)

        self.fig = fig # go.Figure()
        
        self.x_range = np.arange(-0.1, 1.1, h)
        self.y_range = np.arange(-0.1, 1.1, h)

        # creating a mesh to plot decision boundary
        xx, yy = np.meshgrid(self.x_range, self.y_range, indexing='ij')
        Z = np.array([[self.classify([x, y]) for x in self.x_range] for y in self.y_range])
        self.trace_contour = go.Contour(
            z=Z, colorscale=color_scale, x=self.x_range, y=self.y_range
        )
        # using the contourf function to create the plot
        self.fig.add_trace(self.trace_contour)
        
        self.fig.update_layout(title = f"w={self.w}, b={self.b}")
        
        
        display(self.fig)

        
    def _gradient(self, node, exp, output):
        """
        Return the gradient for a weight.
        This is the value of delta-w.
        """
        return node * (exp - output)

    def update_weights(self, exp, output):
        """
        Update weights and bias based on their respective gradients
        """
        for i in range(self.input_nodes):
            self.w[i] += self.lr * self._gradient(self.node_val[i], exp, output)

        # the value of the bias node can be considered as being 1 and the weight between this node
        # and the output node being self.b
        self.b += self.lr * self._gradient(1, exp, output)

    def forward(self, datapoint):
        """
        One forward pass through the perceptron.
        Implementation of "wX + b".
        """
        return self.b + np.dot(self.w, datapoint)

    def classify(self, datapoint):
        """
        Return the class to which a datapoint belongs based on
        the perceptron's output for that point.
        """
        if self.forward(datapoint) >= 0:
            return 1

        return 0
    

    def train(self, max_iters=100):
        """
        Train a single layer perceptron.
        """
        # the number of consecutive correct classifications
        correct_counter = 0

        n_iters = 0 # number of training data points seen
        
        for train, target in cycle(zip(self.train_data, self.target)):
            # end if all points are correctly classified
            #print(f"start {n_iters}")
            if correct_counter == len(self.train_data) or n_iters == max_iters:
                print("quit")
                break

            n_iters += 1
            
            output = self.classify(train)
            self.node_val = train

            if output == target:
                correct_counter += 1
            else:
                # if incorrectly classified, update weights and reset correct_counter
                self.update_weights(target, output)
                correct_counter = 0
        
            if n_iters > 0:
                # Z = np.array([[ 1./ (1. + np.exp(self.forward([x, y])) ) for x in self.x_range] for y in self.y_range])
                Z = np.array([[ self.classify([x, y]) for x in self.x_range] for y in self.y_range])

                #self.fig.update_traces(z=Z)
                #self.fig.update_layout(title = f"w={self.w}, b={self.b}, n_iters={n_iters}")

                #print(f"iter: {n_iters}, w= {self.w}")
                with self.fig.batch_update():
                    self.fig.data[0].z = Z
                    self.fig.layout.title = f"w={self.w}, b={self.b}, n_iters={n_iters}, cumulative success={correct_counter}"
                
                time.sleep(0.02)

## train the XOR function

![XOR function](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*aN7_uKSN8iWUktGOKa1Vgg.png)


In [6]:
p_xor = Perceptron(train_data, target_xor, fig=go.FigureWidget())
p_xor.train( max_iters=500)

FigureWidget({
    'data': [{'colorscale': [[0.0, 'gold'], [0.5, 'mediumturquoise'], [1.0,
                   …

quit


In [ ]:
y= f(x; w, b)

## Questions
- What is maximum cumulative success value? How to interprete this value?
- Cumulative success = 0 means the model get the output for all four inputs wrong. Since the output is either 0 or 1. Can you just "flip" the output value to make the model to work?

## train NAND function

In [8]:
p_nand = Perceptron(train_data, target_nand, fig=go.FigureWidget())
p_nand.train( max_iters=500)

FigureWidget({
    'data': [{'colorscale': [[0.0, 'gold'], [0.5, 'mediumturquoise'], [1.0,
                   …

quit


## train FIRST function

In [11]:
# FIRST function: it takes the "first" component from the 2-element input, x= (a, b) -> a
p_first = Perceptron(train_data, target_first, fig=go.FigureWidget())
p_first.train( max_iters=1500)

FigureWidget({
    'data': [{'colorscale': [[0.0, 'gold'], [0.5, 'mediumturquoise'], [1.0,
                   …

quit


## Questions
- Does above model match your intuition?
- How many different possible model fittings exist? Is it finite or infinite? How do you define the "best" fit?

# Bonus

Interactive visualization -- the following code snipplet can be very handy when we are doing data exploration. They are not dependent on linear regression problems.

## Updating graph in a loop

In [12]:

figg = go.FigureWidget()

# Add some traces
figg.add_trace(go.Scatter(x=[1, 2, 3], y=[4, 5, 6]))
display(figg)  # multiple calls will render multiple images! figg.show() won't animate

# Update the traces in a loop
for i in range(3):
    with figg.batch_update():
        figg.data[0].y = np.sin( (i + 1) * np.array(figg.data[0].x) )

    time.sleep(2)
    print(i)
    

FigureWidget({
    'data': [{'type': 'scatter', 'uid': '8738b6b7-8af7-4f7b-9a74-a5f9d167dc06', 'x': [1, 2, 3],…

0
1
2


## Use widget control to update params

In [13]:
from ipywidgets import interact

fig3 = go.FigureWidget()

fig3.add_trace(go.Scatter(x=[1, 2, 3], y=[4, 5, 6]))

display(fig3)

FigureWidget({
    'data': [{'type': 'scatter', 'uid': '1432612a-372f-4275-95cd-0200267549cf', 'x': [1, 2, 3],…

In [14]:
xs=np.linspace(0, 6, 100)

@interact(a=(1.0, 4.0, 0.01), b=(0, 10.0, 0.01), color=['red', 'green', 'blue'])
def update(a=3.6, b=4.3, color='blue'):
    with fig3.batch_update():
        fig3.data[0].x=xs
        fig3.data[0].y=np.sin(a*xs-b)
        fig3.data[0].line.color=color

interactive(children=(FloatSlider(value=3.6, description='a', max=4.0, min=1.0, step=0.01), FloatSlider(value=…